In [1]:
import tensorflow as tf
import sys
import math
import os
import numpy as np

from sklearn.model_selection import train_test_split
from tqdm import tqdm
from getvector import getvector
from tensorflow.python.platform import gfile
from progress.bar import Bar

In [2]:
if len(sys.argv) <= 1:
    print ('enter option: "train" or "test <image file name>"')
    exit()
    

In [3]:
len(sys.argv)

3

In [4]:
data_inputs = []
data_labels = []

In [5]:
data_inputs = np.loadtxt('data_inputs.txt')
data_labels = np.loadtxt('data_labels.txt')

In [ ]:
image_dir = './Trin'
file_list = []
file_glob = os.path.join(image_dir, '*.jpg')
file_list.extend(gfile.Glob(file_glob))

In [ ]:
# I only used 300 images from the cats and dogs dataset
file_list = file_list[0:300]
bar = Bar('Inception-V3 is processing images:', max=300)

In [ ]:
for file_name in file_list:
    data_inputs.append(getvector(file_name))
    if 'cat' in file_name:
        data_labels.append([1, 0])
    else:
        data_labels.append([0, 1])
    bar.next()
bar.finish()

np.savetxt('data_inputs.txt', data_inputs)
np.savetxt('data_labels.txt', data_labels)

In [6]:
train_inputs, valtest_inputs, train_labels, valtest_labels = train_test_split(data_inputs, data_labels, test_size=0.3, random_state=42)
val_inputs, test_inputs, val_labels, test_labels = train_test_split(valtest_inputs, valtest_labels, test_size=0.4, random_state=43)

In [7]:
n_train = len(train_inputs)

In [24]:
learning_rate = 0.01
batch_size = 64
hm_epochs = 3
log_batch_step = 50

In [9]:
n_features = np.size(train_inputs, 1)
n_labels = np.size(train_labels, 1)


In [10]:
n_labels

2

In [11]:
x = tf.placeholder(tf.float32, (None, n_features))
y = tf.placeholder(tf.float32, (None, n_labels))

In [12]:
weights = tf.Variable(tf.truncated_normal((n_features, n_labels), stddev=0.1), name='weights')
bias = tf.Variable(tf.zeros(n_labels), name='bias')
tf.add_to_collection('vars', weights)
tf.add_to_collection('vars', bias)

In [14]:
logits = tf.add(tf.matmul(x, weights), bias)
prediction = tf.nn.softmax(logits)
tf.add_to_collection('pred', prediction)

In [15]:
difference = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits)
loss = tf.reduce_sum(difference)

In [16]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [25]:
with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            num_batch = int(n_train/batch_size)+1
            for _ in range(num_batch):
                randidx = np.random.randint(n_train, size=batch_size)
                epoch_x,epoch_y = train_inputs[randidx, :],train_labels[randidx, :]
                _,c = sess.run([optimizer,loss], feed_dict = {x:epoch_x,y:epoch_y})
                epoch_loss+=c
            print('Epoch', epoch, 'completed out of', hm_epochs,'loss',epoch_loss)
                
        correct = tf.equal(tf.argmax(prediction,1), tf.argmax(y,1))
        accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
        print('Accuracy:', accuracy.eval({x:test_inputs, y :test_labels}))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 0 completed out of 3 loss 612.936811447
Epoch 1 completed out of 3 loss 311.75328064
Epoch 2 completed out of 3 loss 76.4353916645
Accuracy: 0.888889
